# Introduction to TileMatrixSets

The goal of this notebook is to give a quick introduction to the new TMS option in rio-tiler.

# Requirements

To be able to run this notebook you'll need the following requirements:
- rio-tiler~= 4.0
- ipyleaflet

In [2]:
# !pip install rio-tiler
# !pip install ipyleaflet

In [3]:
import morecantile
from ipyleaflet import (
    Map,
    basemaps,
    basemap_to_tiles,
    TileLayer,
    WMSLayer,
    GeoJSON,
    projections
)

## Data

For this demo we will use some High resolution RGB-Nir data hosted on [AWS](https://registry.opendata.aws/nj-imagery/).

In [4]:
# For this DEMO we will use this file
src_path = "https://njogis-imagery.s3.amazonaws.com/2020/cog/I7D16.tif"

## Tile Server

For this demo, we need to create a minimal tile server.

In [5]:
import json
from concurrent import futures

from tornado import web
from tornado import gen
from tornado.httpserver import HTTPServer
from tornado.concurrent import run_on_executor

from rio_tiler.io import Reader
from rio_tiler.errors import TileOutsideBounds
from rio_tiler.profiles import img_profiles

class TileServer:
    
    def __init__(self, src_path):
        """Initialize Tornado app."""
        self.server = None
        self.app = web.Application([
            (r"^/tiles/(\w+)/(\d+)/(\d+)/(\d+)", TileHandler, {"url": src_path}),
        ])

    def start(self):
        """Start tile server."""
        self.server = HTTPServer(self.app)
        self.server.listen(8080)
  
    def stop(self):
        """Stop tile server."""
        if self.server:
            self.server.stop()


class TileHandler(web.RequestHandler):
    """Tile requests handler."""

    executor = futures.ThreadPoolExecutor(max_workers=16)

    def initialize(self, url):
        """Initialize tiles handler."""
        self.url = url

    @run_on_executor
    def _get_tile(self, tms, z, x, y):

        try:
            with Reader(self.url, tms=morecantile.tms.get(tms)) as cog:
                img = cog.tile(x, y, z, indexes=(1,2,3))
        except TileOutsideBounds:
            raise web.HTTPError(404)

        prof = img_profiles.get("PNG", {})
        return img.render(img_format="PNG", **prof)

    @gen.coroutine
    def get(self, tms, z, x, y):
        """Retunrs tile data and header."""
        self.set_header("Access-Control-Allow-Origin", "*")
        self.set_header("Access-Control-Allow-Methods", "GET")
        self.set_header("Content-Type", "image/png")
        self.set_header("Cache-Control", "no-store, no-cache, must-revalidate")
        res = yield self._get_tile(tms, int(z), int(x), int(y))
        self.write(res)


ts = TileServer(src_path)
ts.start()

List the supported TMS from morecantile

In [6]:
print("Supported TMS:")
for name, tms in morecantile.tms.tms.items():
    print("-", name, "|",tms.crs)

Supported TMS:
- LINZAntarticaMapTilegrid | urn:ogc:def:crs:EPSG::5482
- EuropeanETRS89_LAEAQuad | urn:ogc:def:crs:EPSG::3035
- CanadianNAD83_LCC | urn:ogc:def:crs:EPSG::3978
- UPSArcticWGS84Quad | urn:ogc:def:crs:EPSG::5041
- NZTM2000 | urn:ogc:def:crs:EPSG::2193
- NZTM2000Quad | urn:ogc:def:crs:EPSG::2193
- UTM31WGS84Quad | urn:ogc:def:crs:EPSG::32631
- UPSAntarcticWGS84Quad | urn:ogc:def:crs:EPSG::5042
- WorldMercatorWGS84Quad | urn:ogc:def:crs:EPSG::3395
- WGS1984Quad | urn:ogc:def:crs:EPSG::4326
- WorldCRS84Quad | urn:ogc:def:crs:OGC::CRS84
- WebMercatorQuad | urn:ogc:def:crs:EPSG::3857


## WebMercator TMS

In [9]:
with Reader(src_path) as cog:
    info = cog.info()
    
print(info.dict(exclude_none=True))

bounds = info.bounds
center = ((bounds[1] + bounds[3]) / 2, (bounds[0] + bounds[2]) / 2)
m = Map(center=center, zoom=info.minzoom, basemap={})

layer = TileLayer(
    url="http://127.0.0.1:8080/tiles/WebMercatorQuad/{z}/{x}/{y}",
    min_zoom=info.minzoom,
    max_zoom=info.maxzoom,
    opacity=1,
)
m.add_layer(layer)
m

{'bounds': BoundingBox(left=-74.3095632062702, bottom=40.603994417539994, right=-74.29151245384847, top=40.61775082944064), 'minzoom': 14, 'maxzoom': 19, 'band_metadata': [('b1', {}), ('b2', {}), ('b3', {}), ('b4', {})], 'band_descriptions': [('b1', ''), ('b2', ''), ('b3', ''), ('b4', '')], 'dtype': 'uint16', 'nodata_type': 'None', 'colorinterp': ['red', 'green', 'blue', 'undefined'], 'driver': 'GTiff', 'count': 4, 'overviews': [2, 4, 8, 16], 'width': 5000, 'height': 5000}


Map(center=[40.610872623490316, -74.30053783005934], controls=(ZoomControl(options=['position', 'zoom_in_text'…

## WGS84 TMS

In [ ]:
with Reader(src_path, tms=morecantile.tms.get("WorldCRS84Quad")) as cog:
    info = cog.info()
print(info.dict(exclude_none=True))

bounds = info.bounds
center = ((bounds[1] + bounds[3]) / 2, (bounds[0] + bounds[2]) / 2)
m = Map(center=center, zoom=info.minzoom, basemap={}, crs=projections.EPSG4326)

layer = TileLayer(
    url="http://127.0.0.1:8080/tiles/WorldCRS84Quad/{z}/{x}/{y}",
    min_zoom=info.minzoom,
    max_zoom=info.maxzoom,
    opacity=1,
)
m.add_layer(layer)
m

In [11]:
ts.stop()